<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/incremental_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 11.8 MB/s eta 0:00:00


In [10]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import emoji
from text_preprocessing import cleaning_tweets

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
model = pickle.load(open("/content/model.pkl", "rb"))
vectorizer = pickle.load(open("/content/vectorizer.pkl", "rb"))

In [11]:
data = pd.read_csv("/content/synthetic_sentiment_data.csv")

In [12]:
data.head(1)

,Unnamed: 0,Date,Tweet,class,gpt_label
0,0,2024-01-01T00:00:00,I had the worst experience with their customer...,0,0


In [13]:
data.drop(['Unnamed: 0', 'Date', 'class'], inplace=True, axis=1)

In [14]:
cleaned_tweets = []
emoji_list = []

for i in range(data.shape[0]):
  clean_text, extracted_emojis = cleaning_tweets(str(data.iloc[i]["Tweet"]))
  cleaned_tweets.append(clean_text)
  emoji_list.append(extracted_emojis)

data["clean_tweet"] = cleaned_tweets
data["extracted_emojis"] = emoji_list

In [15]:
def emoji_to_text(emoji_string):
    return ' '.join(emoji.demojize(e) for e in emoji_string) if emoji_string else ''

data["emoji_text"] = data["extracted_emojis"].apply(emoji_to_text)

In [ ]:
X_new, y_new = df_new['tweet'], df_new['sentiment']

X_new_tfidf = vectorizer.transform(X_new)

# Incrementally train model
model.partial_fit(X_new_tfidf, y_new, classes=['positive', 'negative', 'neutral'])

# Save updated model
pickle.dump(model, open("../backend/model/model.pkl", "wb"))

print("Model updated successfully!")
